# ダイクストラ法
SSP(Single Source Shortest Path) 単一始点対最短経路問題を解く方法の一つ。  
### メリット
無向グラフや閉路がある場合でも使える。
### デメリット
負の重みを持つグラフでは使えない => ベルマンフォード法を使う必要がある


## アルゴリズムの流れ
最短距離が確定している部分と繋がっている頂点の中で、最もコストが低くたどり着ける点を選んで、最短距離を確定させる。これを繰り返す

`shortest_distance`に0から各頂点までの最短距離を記録  
`kakutei`で各頂点の最短距離が確定済みかどうかを保存  
`heap_data`に、確定済みの頂点と接する頂点の番号と始点からその頂点に辿り着くまでのコストを保存  

1. 始点(i=0)のデータをheap_dataに入れて、最初の探索対象とする
2. heap_dataから、次に確定させるべき頂点(確定済み頂点と接する頂点の中で、最もコストが低い頂点)の情報を取得
3. 2で取得したデータを最短距離として、確定させる。
4. 3で確定させた頂点から、移動可能な頂点と、移動コストをheap_dataに保存
5. 全ての頂点が確定するまで、2-4を繰り返す

### ポイント
確定済み頂点と接する頂点の中で、最もコストが低い頂点を探す 2 のステップにおいて、計算量削減のため、heapq(優先度付きキュー)を用いている


### C++ での実装
```cpp
struct Edge {
  int to;    // 行き先
  int cost;  // コスト
};
using Graph = vector<vector<Edge>>;
vector<int> Dijkstra(const Graph& graph, int start, int goal) {
  int N = graph.size();
  vector<int> dist(N + 1, 2000000000);
  vector<bool> kakutei(N + 1, false);
  // (コスト、id)のペアを自動的に大きい順に取り出せる
  priority_queue<pii, vector<pii>, greater<pii>> Q;

  // ダイクストラ法
  dist[start] = 0;
  Q.push(make_pair(dist[start], 1));
  while (!Q.empty()) {
    int pos = Q.top().second;
    Q.pop();

    if (kakutei[pos]) continue;

    kakutei[pos] = true;
    for (Edge adj : graph[pos]) {
      int adj_id = adj.to;
      int cost = adj.cost;
      if (dist[adj_id] > dist[pos] + cost) {
        dist[adj_id] = dist[pos] + cost;
        Q.push(make_pair(dist[adj_id], adj_id));
      }
    }
  }
  return dist;
}
int main() {
  int N, M, a, b, c;
  cin >> N >> M;
  Graph G(N + 1);
  rep(i, 0, M) {
    cin >> a >> b >> c;
    G[a].push_back({b, c});  // {b, c} として Edge Struct を省略して宣言している
    G[b].push_back({a, c});
  }
  // 初期化
  vector<int> dist = Dijkstra(G, 1, N);
  rep(i, 1, N + 1) cout << (dist[i] == 2000000000 ? -1 : dist[i]) << endl;
  return 0;
}
```


### Python での実装

In [1]:
import math
from sys import stdin, setrecursionlimit
import heapq

setrecursionlimit(10**6)


def dijkstra(graph):
    N = len(graph)
    shortest_distance = [math.inf]*N
    kakutei = [False]*N
    shortest_distance[0] = 0
    # 現在確定済みの頂点から行けるノードの中で最もコストが低いノードを、heapq(優先度付きキュー)で管理する
    heap_data = []
    # heapqは先頭のデータをkeyとして大小比較を行なってくれる
    heapq.heappush(heap_data, (shortest_distance[0], 0))
    while len(heap_data) > 0:
        curr_cost, pos = heapq.heappop(heap_data)

        if kakutei[pos]:
            continue

        kakutei[pos] = True
        for adj, adj_cost in graph[pos]:
            if shortest_distance[adj] >= curr_cost+adj_cost:
                shortest_distance[adj] = curr_cost+adj_cost
                heapq.heappush(heap_data, (curr_cost+adj_cost, adj))
    return shortest_distance


def main():
    N, M = map(int, stdin.readline().split())
    graph = [[] for _ in range(N)]
    for _ in range(M):
        A, B, C = map(int, stdin.readline().split())
        # [隣接先, cost] と重みを扱うしていく
        graph[A-1].append([B-1, C])
        graph[B-1].append([A-1, C])

    shortest_distance = dijkstra(graph)
    for val in shortest_distance:
        print(val if val != math.inf else -1)
